# NII Trainer: Liver and Tumor Segmentation

This notebook demonstrates how to use the NII Trainer package for medical image segmentation.
The package allows you to train cascaded neural networks on NIfTI data for tasks like
liver and tumor segmentation.

## 1. Import Required Modules

First, let's import the core functions from our package.

In [ ]:
import os
import sys
from pathlib import Path

# Import the main module functions


from main import (
    create_liver_tumor_config,
    run_liver_tumor_segmentation,
    get_default_curriculum_params,
    create_custom_experiment
)

# Import some internal modules for more advanced usage
from nii_trainer.configs.config import TrainerConfig, DataConfig, CascadeConfig
from nii_trainer.utils import Experiment, setup_logger
from nii_trainer.visualization import SegmentationVisualizer

# Verify directories
volume_dir = "volume_pt1"
segmentation_dir = "segmentations"

print(f"Volume directory exists: {os.path.exists(volume_dir)}")
print(f"Segmentation directory exists: {os.path.exists(segmentation_dir)}")
print(f"Number of volume files: {len(list(Path(volume_dir).glob('*.nii')))}")
print(f"Number of segmentation files: {len(list(Path(segmentation_dir).glob('*.nii')))}")

## 2. Basic Usage: Running a Complete Experiment

The simplest way to use the package is with the high-level `run_liver_tumor_segmentation` function,
which handles the entire pipeline from data preprocessing to model training and evaluation.

In [ ]:
# Running a basic experiment - this will process data, train, and evaluate
# Note: This can take a long time to run, especially on CPU
# You can set process_data=False if you've already processed the data

experiment = run_liver_tumor_segmentation(
    volume_dir="../volume_pt1",
    segmentation_dir="../segmentations",
    output_dir="../experiments",
    experiment_name="liver_tumor_test",
    process_data=True,        # Set to False to skip data processing
    force_overwrite=False,    # Set to True to force reprocessing
    use_curriculum=True,      # Use curriculum learning approach
     img_size=(512, 512),      # Target image size
     batch_size=16,            # Batch size for training
     slice_step=1,             # Use every slice
     skip_empty=False,         # Include slices without annotations
     epochs=100                # Number of training epochs
 )

# Uncomment the above code to run a full experiment

## 3. Data Processing Only

If you want to just process the NIfTI data without training a model, 
you can create an experiment and call the `process_data` method directly.

In [ ]:
# Create a configuration
config = create_liver_tumor_config(
    volume_dir="volume_pt1",
    img_size=(512, 512),
    window_width=180,
    window_level=50,
    skip_empty=False,
    slice_step=1
)

# Create an experiment
experiment = create_custom_experiment(
    config=config,
    experiment_name="data_processing_only",
    base_dir="experiments"
)

# Process just a few volumes for demonstration (set to None for all)
# The code below is commented out to prevent accidental execution
# Since it can take a while to process all the volumes

# experiment.process_data(
#     volume_dir="volume_pt1",
#     segmentation_dir="segmentations",
#     force_overwrite=False  # Set to True to reprocess existing data
# )

## 4. Creating Custom Configurations

You can customize the training configuration by modifying parameters
or creating a completely custom configuration.

In [ ]:
# Create a custom configuration with modified parameters
custom_config = create_liver_tumor_config(
    volume_dir="volume_pt1",
    output_dir="processed_data_custom",
    img_size=(384, 384),  # Different image size
    batch_size=8,         # Smaller batch size
    window_width=200,     # Different window width
    window_level=40,      # Different window level
    slice_step=2,         # Take every other slice
    train_val_test_split=(0.7, 0.15, 0.15),  # Different split ratio
    learning_rate=5e-5,   # Lower learning rate
    epochs=150            # More epochs
)

# Print the configuration to verify
print(f"Image size: {custom_config.data.img_size}")
print(f"Batch size: {custom_config.data.batch_size}")
print(f"Window parameters: {custom_config.data.window_params}")
print(f"Slice step: {custom_config.data.slice_step}")
print(f"Learning rate: {custom_config.training.learning_rate}")
print(f"Epochs: {custom_config.training.epochs}")

## 5. Working with Curriculum Learning

Curriculum learning is a training strategy where the model first learns simpler tasks
before progressing to more complex ones. In our case, this means first learning to
segment the liver, then learning to segment tumors within the liver.

In [ ]:
# Get default curriculum parameters
curriculum_params = get_default_curriculum_params()
print("Default curriculum parameters:")
print(f"Stage schedule: {curriculum_params['stage_schedule']}")
print(f"Learning rates: {curriculum_params['learning_rates']}")
print(f"Stage freezing: {curriculum_params['stage_freezing']}")

# Create custom curriculum parameters
custom_curriculum = {
    'stage_schedule': [
        (0, 30),  # Train liver stage for 30 epochs
        (1, 70)   # Add tumor stage for 70 more epochs
    ],
    'learning_rates': [2e-3, 1e-4],  # Different learning rates
    'stage_freezing': [False, True]  # Same freezing strategy
}

print("\nCustom curriculum parameters:")
print(f"Stage schedule: {custom_curriculum['stage_schedule']}")
print(f"Learning rates: {custom_curriculum['learning_rates']}")
print(f"Stage freezing: {custom_curriculum['stage_freezing']}")

## 6. Advanced Usage: Working with the Experiment API

For more control over the training process, you can use the Experiment API directly.
This allows you to access the model, trainer, and datasets for custom analysis.

In [ ]:
# Create experiment with custom configuration
experiment = create_custom_experiment(
    config=custom_config,
    experiment_name="advanced_example",
    base_dir="experiments"
)

# Process data (commented out to prevent accidental execution)
# experiment.process_data(
#     volume_dir="volume_pt1",
#     segmentation_dir="segmentations",
#     force_overwrite=False
# )

# Setup data pipeline
# datasets, dataloaders = experiment.setup_data_pipeline()

# Setup model
# model, trainer = experiment.setup_model()

# Train model (with or without curriculum)
# experiment.train(curriculum=True, curriculum_params=custom_curriculum)

# Evaluate model
# metrics = experiment.evaluate()

# Generate visualizations
# experiment.visualize()

## 7. Visualization

After training, you can visualize the segmentation results using the SegmentationVisualizer.

In [ ]:
# Load an experiment (assuming you've already trained one)
# Replace "liver_tumor_test" with your experiment name

# from nii_trainer.utils import Experiment
# experiment_name = "liver_tumor_test"
# saved_experiment = Experiment(
#     config=create_liver_tumor_config(experiment_name=experiment_name),
#     experiment_name=experiment_name,
#     base_dir="experiments"
# )
# 
# # Setup the data pipeline to load the datasets
# datasets, dataloaders = saved_experiment.setup_data_pipeline()
# 
# # Setup the model to load the trained weights
# model, trainer = saved_experiment.setup_model()
# 
# # Create a visualizer
# visualizer = SegmentationVisualizer(
#     model=model,
#     dataloader=dataloaders["val"],
#     class_names=["background", "liver", "tumor"],
#     device="cuda" if torch.cuda.is_available() else "cpu"
# )
# 
# # Display a few examples
# visualizer.display_batch(num_samples=4)

## 8. Customizing Data Processing

For more control over data processing, you can use the BatchProcessor directly.

In [ ]:
# Import the batch processor
from nii_trainer.data import BatchProcessor

# Create a custom processor with specific parameters
custom_processor = BatchProcessor(
    img_size=(512, 512),
    window_params={"window_width": 180, "window_level": 50},
    skip_empty=True,  # Skip empty slices
    slice_step=2,     # Take every other slice
    train_val_test_split=(0.8, 0.1, 0.1),  # Different split ratios
    segmentation_pattern="segmentation-{}.nii",  # Pattern for segmentation files
    volume_pattern="volume-{}.nii"  # Pattern for volume files
)

# Process only a specific subset of volumes (commented out to prevent execution)
# processed_pairs = custom_processor.process_batch(
#     volume_dir="volume_pt1",
#     segmentation_dir="segmentations",
#     output_dir="custom_processed_data",
#     max_volumes=5,  # Only process the first 5 volumes
#     force_overwrite=False
# )

## 9. Conclusion

This notebook demonstrated the key features of the NII Trainer package for medical image segmentation.
You can customize the configuration, preprocessing, training strategy, and visualization to suit your needs.

For more details, refer to the package documentation and source code.